In [8]:
import scanpy as sc

from scprint import scPrint

from scdataloader import Preprocessor
from scdataloader.utils import load_genes, translate

import numpy as np
import pandas as pd
from anndata import AnnData
import matplotlib.pyplot as plt
import scipy.sparse
from collections import Counter

%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')


→ connected lamindb: jkobject/scprint2


## Loading the data

## https://www.nature.com/articles/s41592-020-0935-4#data-availability

In [1]:
! mkdir -p /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/data/temp
! wget  https://www.ncbi.nlm.nih.gov/geo/download/\?acc=GSE141851\&format=file
! mv index.html\?acc=GSE141851\&format=file /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/data/temp/GSE141851.tar
! cd /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/data/temp/ && tar xf GSE141851.tar

--2025-01-21 17:31:40--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE141851&format=file
Resolving maestro-squid.maestro.pasteur.fr (maestro-squid.maestro.pasteur.fr)... 192.168.148.66
Connecting to maestro-squid.maestro.pasteur.fr (maestro-squid.maestro.pasteur.fr)|192.168.148.66|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 172615680 (165M) [application/x-tar]
Saving to: ‘index.html?acc=GSE141851&format=file’

index.html?acc=GSE1 100%[===================>] 164.62M  15.4MB/s    in 11s     

2025-01-21 17:31:52 (14.4 MB/s) - ‘index.html?acc=GSE141851&format=file’ saved [172615680/172615680]



In [19]:
# Create list of all files
files = ! ls /pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/data/temp/GSM451*

# Read and concatenate all files into one AnnData object
adatas_T = []
adatas_C = []
for i, file in enumerate(files[:]):
    # Extract metadata from filename
    basename = file.split('/')[-1].replace('.txt.gz','')
    gsm = basename.split('_')[0]
    condition = '_'.join(basename.split('_')[1:-1])
    if "Neu-Kcl" in condition:
        time = int(condition.split('-')[-1][:-3])
        condition = "neuron"
    elif "mESC" in condition:
        if "WT" in condition:
            time = 0
            condition = "WT"
        else:
            time = condition.split('-')[-1]
            condition = "mESC"
            if "hr" in time:
                time = 60*int(time[:-2])
            else:
                time = int(time[:-3])
    else:
        continue
        
    data_type = basename.split('_')[-1]  # C or T
    
    # Read data
    df = pd.read_csv(file, sep='\t', compression='gzip')
    if data_type == 'T':
        adata_T = AnnData(df.T)
        # Add metadata
        adata_T.obs['gsm'] = gsm
        adata_T.obs['condition'] = condition
        adata_T.obs['time'] = time
        adata_T.obs['data_type'] = data_type
        # Convert to sparse format
        adata_T.X = scipy.sparse.csr_matrix(adata_T.X)
        adatas_T.append(adata_T)
    else:
        adata_C = AnnData(df.T)
        adata_C.obs['gsm'] = gsm
        adata_C.obs['condition'] = condition
        adata_C.obs['data_type'] = data_type
        adata_C.obs['time'] = time
        adata_C.X = scipy.sparse.csr_matrix(adata_C.X)
        adatas_C.append(adata_C)

adata_T = sc.concat(adatas_T)
adata_C = sc.concat(adatas_C)

ValueError: invalid literal for int() with base 10: 'r'

In [ ]:
set([i.split('_')[-2] for i in files])

{'K562-4SU',
 'K562-4SU-N9',
 'K562-4SU-TFEA',
 'K562-4SU-TFEA-N9',
 'K562-NTC-TFEA',
 'K562-NTC-TFEA-N9',
 'Neu-Kcl-0min',
 'Neu-Kcl-120min',
 'Neu-Kcl-15min',
 'Neu-Kcl-30min',
 'Neu-Kcl-60min',
 'mESC-TetTKO-rep1',
 'mESC-TetTKO-rep2',
 'mESC-WT-rep1',
 'mESC-WT-rep2',
 'mESC-b1-Chase-0min',
 'mESC-b1-Chase-12hr',
 'mESC-b1-Chase-24hr',
 'mESC-b1-Chase-30min',
 'mESC-b1-Chase-3hr',
 'mESC-b1-Chase-60min',
 'mESC-b1-Chase-6hr',
 'mESC-b2-Chase-0min',
 'mESC-b2-Chase-12hr',
 'mESC-b2-Chase-24hr',
 'mESC-b2-Chase-30min',
 'mESC-b2-Chase-3hr',
 'mESC-b2-Chase-60min',
 'mESC-b2-Chase-6hr'}

In [28]:
adatas_C[0].X.sum(1)

array([1707, 3315, 2475, ...,   39,   47,   43])

In [31]:
adatas_T[3].X

array([[0, 0, 0, ..., 0, 2, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
adata = sc.concat(adatas_T)
adata.layers['new'] = sc.concat(adatas_C).X
adata.write_h5ad('/pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/data/temp/scntseq.h5ad')
del adatas_C, adatas_T


KeyboardInterrupt: 

In [ ]:
b

In [ ]:
b

In [ ]:
b

In [ ]:
b